In [1]:
import numpy as np # needed (don't change it)
import importlib
import os
import socket
import sys
import setproctitle
import torch

import datetime
import uuid
from argparse import ArgumentParser

from datasets import NAMES as DATASET_NAMES  # nopep8
from datasets import ContinualDataset, get_dataset  # nopep8
from models import get_all_models, get_model  # nopep8

from utils.args import add_management_args
from utils.best_args import best_args
from utils.conf import set_random_seed
from utils.continual_training import train as ctrain
from utils.distributed import make_dp
from utils.training import train
from torchvision import datasets, transforms
import random
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False  


seed_value = 0
set_seed(seed_value)

In [2]:
import argparse

args = argparse.Namespace(
    seed=0, 
    notes=None, 
    non_verbose=1, 
    disable_log=0, 
    validation=0, 
    ignore_other_metrics=1, 
    debug_mode=0, 
    nowand=1, 
    wandb_entity='regaz', 
    wandb_project='mammoth', 
    dataset='seq-cifar10', 
    model='er', 
    lr=0.1, 
    optim_wd=0.0, 
    optim_mom=0.0, 
    optim_nesterov=0, 
    n_epochs=50, 
    batch_size=32, 
    distributed='dp', 
    poisoning_type=1, 
    poisoning_severity=5, 
    n_slots=None, 
    n_poisonings=1, 
    max_classes_per_poisoning=0, 
    sequential_poisonings=False, 
    buffer_size=5000, 
    minibatch_size=32, 
    buffer_mode='reservoir', 
    conf_jobnum='c41d7384-6e52-4d1d-8799-7683921336e2', 
    conf_timestamp='2024-10-23 22:05:53.528238', 
    conf_host='DESKTOP-7A'
)

In [3]:


# Download and transform CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Convert dataset from Tensor to NumPy format
train_data = np.array(trainset.data)  # CIFAR10 dataset's 'data' is already NumPy
train_labels = np.array(trainset.targets)

test_data = np.array(testset.data)
test_labels = np.array(testset.targets)

# Function to split the dataset based on class labels
def get_class_indices(labels, class_range):
    """Return indices in the dataset that belong to the specified class range."""
    indices = [i for i, label in enumerate(labels) if label in class_range]
    return indices

def create_task_datasets(data, labels, task_classes):
    """Create task datasets split by class and return the split data and labels."""
    task_datasets = []
    for class_range in task_classes:
        indices = get_class_indices(labels, class_range)
        task_data = data[indices]
        task_labels = labels[indices]
        task_datasets.append({'data': task_data, 'labels': task_labels})
    return task_datasets

# Define the class ranges for each task
task_classes = [
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7],
    [8, 9]
]

# Create task datasets for both trainset and testset
train_task_datasets = create_task_datasets(train_data, train_labels, task_classes)
test_task_datasets = create_task_datasets(test_data, test_labels, task_classes)

# Verify each task's dataset size
for i, (train_task, test_task) in enumerate(zip(train_task_datasets, test_task_datasets)):
    print(f"Task {i + 1}:")
    print(f"Trainset size: {train_task['data'].shape[0]} | Testset size: {test_task['data'].shape[0]}")

# Example: Access data and labels for the first train and test task
train_task_data, train_task_labels = train_task_datasets[0]['data'], train_task_datasets[0]['labels']
test_task_data, test_task_labels = test_task_datasets[0]['data'], test_task_datasets[0]['labels']

print("First task train data shape:", train_task_data.shape)
print("First task test data shape:", test_task_data.shape)





Files already downloaded and verified
Files already downloaded and verified
Task 1:
Trainset size: 10000 | Testset size: 2000
Task 2:
Trainset size: 10000 | Testset size: 2000
Task 3:
Trainset size: 10000 | Testset size: 2000
Task 4:
Trainset size: 10000 | Testset size: 2000
Task 5:
Trainset size: 10000 | Testset size: 2000
First task train data shape: (10000, 32, 32, 3)
First task test data shape: (2000, 32, 32, 3)


In [4]:

for i in range(5):
    print(np.unique(train_task_datasets[i]['labels']))
    print(np.unique(test_task_datasets[i]['labels']))

[0 1]
[0 1]
[2 3]
[2 3]
[4 5]
[4 5]
[6 7]
[6 7]
[8 9]
[8 9]


In [5]:
from scipy.fft import fft2, fftshift, ifftshift, ifft2
from torch.utils.data import Dataset

class dataset_transform(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = torch.from_numpy(y).type(torch.LongTensor)
        self.transform = transform  # save the transform

    def __len__(self):
        return len(self.y)#self.x.shape[0]  # return 1 as we have only one image

    def __getitem__(self, idx):
        # return the augmented image
        if self.transform:
            x = self.transform(self.x[idx])
        else:
            x = self.x[idx]

        return x.float(), self.y[idx]

class CustomDataset(Dataset):
    def __init__(self, data, labels, extra_labels, transform=None):
        self.data = data
        self.labels = torch.from_numpy(labels).type(torch.LongTensor)
        self.origin = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        original_x = self.origin[index]

        # Convert y and extra_y to LongTensor if they are numpy arrays

        if self.transform:
            x = self.transform(x)

        return x, y, original_x

def poison_transform(img, poisoning_strategy=0, factor=0.5):
    # Skip poisoning if random factor condition is not met
    if random.random() > factor:
        return img
    
    if poisoning_strategy == 0:  # Random noise
        noise = np.random.randn(*img.shape) * 0.2
        return img + noise

    elif poisoning_strategy == 1:  # Pixel permutation
        img_flat = img.flatten()
        np.random.shuffle(img_flat)  # Shuffle pixels randomly
        return img_flat.reshape(img.shape)

    elif poisoning_strategy == 2:  # Low frequencies
        return apply_frequency_filter(img, mode='low')

    elif poisoning_strategy == 3:  # High frequencies
        return apply_frequency_filter(img, mode='high')

    elif poisoning_strategy == 4:  # Add square
        return add_square(img)

    elif poisoning_strategy == 5:  # Vertical flip
        return np.flipud(img)  # Flip image vertically

    else:
        return img

def apply_frequency_filter(img, mode='low'):
    # Apply Fourier transform and shift frequencies
    img_fft = fft2(img)
    img_fft_shifted = fftshift(img_fft)

    rows, cols = img.shape[-2], img.shape[-1]
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols))

    # Create a mask for low or high frequencies
    if mode == 'low':
        mask[crow - 30:crow + 30, ccol - 30:ccol + 30] = 1
    else:  # High frequencies
        mask[:crow - 30, :] = 1
        mask[crow + 30:, :] = 1
        mask[:, :ccol - 30] = 1
        mask[:, ccol + 30:] = 1

    # Apply the frequency filter and inverse transform
    img_fft_filtered = img_fft_shifted * mask
    img_fft_ishifted = ifftshift(img_fft_filtered)
    img_filtered = ifft2(img_fft_ishifted).real

    return img_filtered

def add_square(img):
    # Add a random square patch with random color
    rows, cols = img.shape[-2], img.shape[-1]
    
    # Ensure square size doesn't exceed image size
    if rows < 5 or cols < 5:
        return img  # Skip if the image is too small for a square
    
    square_size = random.randint(5, min(10, rows, cols))  # Dynamically adjust square size
    color = random.random()

    start_row = random.randint(0, rows - square_size)
    start_col = random.randint(0, cols - square_size)

    # Set the square patch to a random color
    img[start_row:start_row + square_size, start_col:start_col + square_size] = color
    return img

def apply_poisoning(x_train, poisoning_strategy=0, factor=1.0):
    # Apply poisoning strategy to each image in the dataset using vectorized operation
    return np.array([poison_transform(img, poisoning_strategy=poisoning_strategy, factor=factor) for img in x_train])


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.sum = 0
        self.count = 0

    def update(self, val, n):
        self.sum += val * n
        self.count += n

    def avg(self):
        if self.count == 0:
            return 0
        return float(self.sum) / self.count

In [6]:
import tqdm
from backbone.ResNet18 import resnet18
import torch.nn.functional as F
import torchvision.transforms as transforms


TRANSFORM = transforms.Compose([
        transforms.ToTensor(),
    ])

backbone = resnet18(10)
loss = F.cross_entropy

model = get_model(args, backbone, loss,  transform = transforms.Compose(
            [transforms.ToPILImage(), TRANSFORM]))
# model = get_model(args, backbone, loss,  transform = transforms.Compose(
#             [TRANSFORM]))
print(model.__class__.__name__)
# Set a seed for reproducibility

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

## train process
for task, train_data in enumerate(train_task_datasets):
    model.net.train()
    x_train, y_train = train_data['data'], train_data['labels']
    ##################poisoning#############################
    
    if task ==2:
        poisoning_strategy ={0: 'Random noise', 
                            1: 'Pixel permutation', 
                            2: 'Low frequencies', 
                            3: 'High frequencies', 
                            4: 'Add square', 
                            5: 'Vertical flip'}
        print('poisoning the data with {}'.format(poisoning_strategy[0]))
        x_train_poison = []
        for x in x_train:
            x_poison = poison_transform(x, poisoning_strategy=0, factor=0.5)
            x_train_poison.append(x_poison)
        x_train_poison = np.array(x_train_poison)
        x_train = x_train_poison
    #####################################################
    for epoch in tqdm.tqdm(range(50)):
        train_dataset = dataset_transform(x_train, y_train, transform=transforms.Compose([
        transforms.ToTensor(),
        ]))
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=False)
        model.net.train()
        loss_meter = AverageMeter()
        acc_meter = AverageMeter()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            loss = model.meta_observe(inputs, labels, inputs)

    # if task == 2:
    #     break
        
    #eval
    model.net.eval()
    acc_array = np.zeros(len(test_task_datasets))
    with torch.no_grad():
        for task, test_data in enumerate(test_task_datasets):
            acc = AverageMeter()
            x_test, y_test = test_data['data'], test_data['labels']
            test_dataset = dataset_transform(x_test, y_test, transform=transforms.Compose([
                transforms.ToTensor(),
            ]))
            test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=500, shuffle=False)
            # acc_meter = AverageMeter()
            for i, (inputs, labels) in enumerate(test_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model.net(inputs)
                _, preds = torch.max(outputs, 1)
                correct_cnt = (preds == labels).sum().item() / inputs.size(0) 

                acc.update(correct_cnt, inputs.size(0))
            acc_array[task] = acc.avg()
    print(acc_array)




    





Er


 12%|█▏        | 6/50 [00:46<05:41,  7.76s/it]


KeyboardInterrupt: 

: 

In [7]:
indices = torch.FloatTensor(50).uniform_(0, 100).long()

In [8]:
print(indices)

tensor([18, 44,  6, 91, 33, 71, 38, 50, 89, 45, 81, 60, 57, 36, 62, 43, 20, 52,
        11, 13, 94, 64, 93, 86, 94, 90,  6, 19, 80, 18,  4, 78, 86, 30,  1, 59,
        30, 32,  5, 48, 60,  7, 32, 90, 43,  9,  1, 88, 96, 31])


In [9]:
place_left = max(0, 50)
print(place_left)

50


In [10]:
indices = torch.FloatTensor(5).uniform_(0, 20).long()
print(indices)

tensor([13, 16,  7, 16, 15])


In [11]:
valid_indices = (indices < 10).long()
print(valid_indices)
idx_new_data = valid_indices.nonzero().squeeze(-1)
idx_buffer   = indices[idx_new_data]
print(idx_new_data)
print(idx_buffer)
idx_map = {idx_buffer[i].item(): idx_new_data[i].item() for i in range(idx_buffer.size(0))}
print(idx_map)

tensor([0, 0, 1, 0, 0])
tensor([2])
tensor([7])
{7: 2}
